# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
print('total = ' + str(len(df)))
print(df.group.value_counts())
print(df.action.value_counts())
print('Number of unique user ids: ' + str(len(df.id.unique())))

total = 8188
control       4264
experiment    3924
Name: group, dtype: int64
view     6328
click    1860
Name: action, dtype: int64
Number of unique user ids: 6328


Does the same user id view twice?  Same if one or more user id's don't view at all.

In [3]:
len(df[df['action']=='view'].id.unique())

6328

In [4]:
print(len(df[df['group']=='control'].id.unique()))
print(len(df[df['group']=='experiment'].id.unique()))

3332
2996


Does there exists two records with the same id having different value for the group?  No. See below!

In [5]:
df.groupby('id').agg('|'.join).group.value_counts()

control                  2400
experiment               2068
control|control           932
experiment|experiment     928
Name: group, dtype: int64

In [6]:
df.groupby('id').agg('|'.join).action.value_counts()

view          4468
view|click    1860
Name: action, dtype: int64

In [7]:
df.groupby('id').agg('|'.join).head()

,timestamp,group,action
id,,,
182988,2016-11-26 14:38:47.682931,experiment,view
182994,2016-12-14 13:19:00.940677|2016-12-14 13:20:29...,control|control,view|click
183089,2016-09-29 23:04:42.722325,control,view
183136,2016-12-27 10:13:58.808014,experiment,view
183141,2016-11-20 07:58:52.768641|2016-11-20 08:00:22...,experiment|experiment,view|click


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [8]:
import scipy
import statsmodels
import statsmodels.stats
import statsmodels.stats.proportion
n0 = 2400 + 932
n1 = 2068 + 928
k0 = 932
k1 = 928
zstat, pvalue = statsmodels.stats.proportion.proportions_ztest([k0,k1],[n0,n1],value=0)
print(zstat)
print(pvalue)

-2.618563885349469
0.008830075576595804


In [9]:
phat = (k0 + k1)/(n0 + n1)
zstat_pool, pvalue_pool = statsmodels.stats.proportion.proportions_ztest([k0,k1],[n0,n1],value=0,
                                                                          prop_var=phat)
print(zstat_pool)
print(pvalue_pool)

-2.618563885349469
0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
n1*k0/n0

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
zstat

-2.618563885349469

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
pvalue

0.008830075576595804

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.